In [ ]:
%reload_ext autoreload
%autoreload 2
import re
from pathlib import Path

import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
from scipy.optimize import lsq_linear
from ufoldy.piecewisefunction import PCF, PLF
from ufoldy.reactionrate import ReactionRate
from ufoldy.ufoldy import ufoldy

# Input reference spectrum

In [ ]:
NG = 1102
phi_ref_data = np.loadtxt("Spectra/1102_PWR-UO2-15.txt")
phi_x = phi_ref_data[: NG + 1][::-1]
phi_y = np.concatenate((phi_ref_data[NG + 1 :][::-1], np.array([0])))
phi_ref = PCF(phi_x, phi_y, normalize=False)  # normvalue=1e13

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(16, 15))

energy_range = np.logspace(np.log10(phi_ref.x[0]), np.log10(phi_ref.x[-1]), 100001)

# axs[0].loglog(phi_ref.x, phi_ref.y, "o")
axs[0].loglog(energy_range, phi_ref(energy_range))

axs[0].grid(True)
axs[0].set_xlabel("Energy (eV)")
axs[0].set_ylabel("Flux")
axs[0].set_title("PWR UO2 15 MWd/tHM")
axs[0].set_xlim([1e-3, 1e8])
# axs[0].set_ylim([1e-12, 1e-6])

axs[1].grid(True)
axs[1].semilogx(energy_range, phi_ref(energy_range))
axs[1].set_xlim([1e-3, 1e8])

plt.show()

# Get some cross sections and calculate the "correct" reaction rates

In [ ]:
# Assume all cross sections are in xsdata folder and all start with ENDF_Sig

p = Path("Data").glob("**/ENDF_Sig_*.csv")
xsfiles = [x for x in p if x.is_file()]

reactions = []

for file in xsfiles:
    # Get xs data from file
    reaction = re.match(r"[\w]*\\ENDF_Sig_([\w']*).csv", str(file)).group(1)
    xsdata = np.loadtxt(file, delimiter=";", skiprows=3)

    # Create the ReactionRate object
    r = ReactionRate(reaction, xsdata[:, 0], xsdata[:, 1])

    # Calculate the reaction rate using the reference spectrum
    r.reaction_rate = r.cross_section.convolute(phi_ref)
    print(f"{r.name:20s}: {r.reaction_rate:20.16e}")

    reactions.append(r)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(16, 10))

for r in reactions:
    axs.loglog(energy_range, r.cross_section(energy_range), label=r.name)

axs.grid(True)
axs.legend(loc="best")
axs.set_ylim([1e-12, 1e6])
plt.show()

In [ ]:
# ig_x = np.logspace(3, np.log10(2e8),11)
# ig_y = 4e-4*np.ones_like(ig_x)

initial_guess = PCF.flat(1e3, 2e8, 1e-4)
# initial_guess = PCF(ig_x,ig_y)
max_prior_factor = 10.0
gradient_weight = 1e8
refinements = 10
max_iter = 20
tol_iter = 1e-2
lb = 1e-10
ub = 1
verbose = True

In [ ]:
#%%timeit
fluxes, indices = ufoldy(
    reactions,
    initial_guess,
    max_prior_factor=max_prior_factor,
    gradient_weight=gradient_weight,
    refinements=refinements,
    tol_iter=tol_iter,
    max_iter=max_iter,
    lbound=lb,
    ubound=ub,
    verbose=verbose,
)

In [ ]:
refinement = 5
istart = np.sum(indices[0 : refinement - 1], dtype=int)
iend = np.sum(indices[0:refinement])
#plotfluxes = fluxes[istart:iend]
plotfluxes = [fluxes[istart], fluxes[iend-1]]

fig, axs = plt.subplots(4, 1, figsize=(16, 15))

energy_range = np.logspace(-6, 8.5, 10001)

axs[0].loglog(energy_range, phi_ref(energy_range))
axs[0].grid(True)
axs[0].set_ylabel("Flux")
axs[0].set_title("PWR UO2 15 MWd/tHM")

axs[1].semilogx(energy_range, phi_ref(energy_range))
axs[1].set_ylabel("Flux")
axs[1].grid(True)
for i, f in enumerate(plotfluxes):
    axs[0].loglog(f.x, f.y, "o", color=f"C{i+1}")
    axs[0].loglog(energy_range, f(energy_range), color=f"C{i+1}", label=f"i={i}")
    axs[1].semilogx(f.x, f.y, "o", color=f"C{i+1}")
    axs[1].semilogx(energy_range, f(energy_range), color=f"C{i+1}", label=f"i={i}")


axs[0].legend(loc="best")
axs[1].legend(loc="best")


axs[2].loglog(energy_range, np.abs(f(energy_range)-phi_ref(energy_range))/phi_ref(energy_range))
axs[2].set_ylabel("Relative error")
axs[2].grid(True)

axs[3].semilogx(energy_range, np.abs(f(energy_range)-phi_ref(energy_range)))
axs[3].set_ylabel("Absolute error")
axs[3].grid(True)

axs[-1].set_xlabel("Energy (eV)")

plt.show()